In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [18]:
df=pd.read_csv('/content/cleaned_network_data.csv')
df.head()

,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,conn_state,...,dns_rcode,dns_AA,dns_RD,dns_RA,dns_rejected,http_request_body_len,http_response_body_len,http_status_code,label,type
0,192.168.1.37,4444,192.168.1.193,49178,tcp,-,290.371539,101568,2592,OTH,...,0,none,none,none,none,0,0,0,1,backdoor
1,192.168.1.193,49180,192.168.1.37,8080,tcp,-,0.000102,0,0,REJ,...,0,none,none,none,none,0,0,0,1,backdoor
2,192.168.1.193,49180,192.168.1.37,8080,tcp,-,0.000148,0,0,REJ,...,0,none,none,none,none,0,0,0,1,backdoor
3,192.168.1.193,49180,192.168.1.37,8080,tcp,-,0.000113,0,0,REJ,...,0,none,none,none,none,0,0,0,1,backdoor
4,192.168.1.193,49180,192.168.1.37,8080,tcp,-,0.000130,0,0,REJ,...,0,none,none,none,none,0,0,0,1,backdoor


In [19]:
import numpy as np
unique, counts = np.unique(y, return_counts=True)
for label, count in zip(le.inverse_transform(unique), counts):
    print(f"Class '{label}' has {count} samples")


Class 'backdoor' has 18711 samples
Class 'ddos' has 19993 samples
Class 'dos' has 18992 samples
Class 'injection' has 19964 samples
Class 'mitm' has 1041 samples
Class 'normal' has 42021 samples
Class 'password' has 19861 samples
Class 'ransomware' has 14735 samples
Class 'scanning' has 20000 samples
Class 'xss' has 15137 samples


In [20]:
# Find and remove rows where the original 'type' column is NaN or 'nan'
df = df[df['type'].notna()]                    # Remove actual NaNs
df = df[df['type'].str.lower() != 'nan']       # Remove string 'nan' if present


In [21]:

from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# Drop columns that are not useful for training and are not present in the dataframe
columns_to_drop = ['Flow ID', 'Source IP', 'Destination IP', 'Timestamp']
existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]
df = df.drop(existing_columns_to_drop, axis=1)

# Separate features (X) and target (y)
X = df.drop('type', axis=1)
y = df['type']

# Convert target variable to numerical labels if it's categorical
le = LabelEncoder()
y = le.fit_transform(y)



In [22]:
# Convert object columns to category dtype
for col in X.select_dtypes(include='object').columns:
    X[col] = X[col].astype('category')


In [13]:
# Split again after conversion
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Re-initialize and train model
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', enable_categorical=True)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=le.classes_)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)


Accuracy: 0.9964393854435348
Classification Report:
              precision    recall  f1-score   support

    backdoor       1.00      1.00      1.00      3742
        ddos       0.99      0.99      0.99      3999
         dos       1.00      0.99      0.99      3799
   injection       0.99      0.99      0.99      3993
        mitm       0.94      0.96      0.95       208
      normal       1.00      1.00      1.00      8404
    password       1.00      1.00      1.00      3972
  ransomware       1.00      1.00      1.00      2947
    scanning       0.99      1.00      0.99      4000
         xss       1.00      1.00      1.00       604

    accuracy                           1.00     35668
   macro avg       0.99      0.99      0.99     35668
weighted avg       1.00      1.00      1.00     35668



In [14]:
import joblib

# Save the trained model
joblib.dump(model, 'xgboost_model.pkl')


['xgboost_model.pkl']

In [15]:
from google.colab import files

# Download the model file
files.download('xgboost_model.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
from sklearn.preprocessing import LabelEncoder

# Encode object or category columns
for col in X.select_dtypes(include=['object', 'category']).columns:
    le_col = LabelEncoder()
    X[col] = le_col.fit_transform(X[col])

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train RandomForest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=le.classes_)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)




Accuracy: 0.9959045443805623
Classification Report:
              precision    recall  f1-score   support

    backdoor       1.00      1.00      1.00      3742
        ddos       0.99      0.99      0.99      3999
         dos       0.99      0.99      0.99      3798
   injection       0.99      0.99      0.99      3993
        mitm       0.93      0.91      0.92       208
      normal       1.00      1.00      1.00      8404
    password       1.00      1.00      1.00      3972
  ransomware       1.00      1.00      1.00      2947
    scanning       0.99      0.99      0.99      4000
         xss       1.00      1.00      1.00      3028

    accuracy                           1.00     38091
   macro avg       0.99      0.99      0.99     38091
weighted avg       1.00      1.00      1.00     38091



In [25]:
# Save model
joblib.dump(rf_model, 'random_forest_model.pkl')

# Download model file
files.download('random_forest_model.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [26]:
# Get predictions
xgb_preds = model.predict(X_test)
rf_preds = rf_model.predict(X_test)

# Decode back to class labels
xgb_labels = le.inverse_transform(xgb_preds)
rf_labels = le.inverse_transform(rf_preds)
true_labels = le.inverse_transform(y_test)

# Create a comparison DataFrame
import pandas as pd

comparison_df = pd.DataFrame({
    'True Label': true_labels,
    'XGBoost Prediction': xgb_labels,
    'Random Forest Prediction': rf_labels
})

# Show the first few rows
print(comparison_df.head(20))


   True Label XGBoost Prediction Random Forest Prediction
0      normal             normal                   normal
1        ddos               ddos                     ddos
2        mitm               mitm                     mitm
3        ddos               ddos                     ddos
4    backdoor           backdoor                 backdoor
5      normal             normal                   normal
6        ddos               ddos                     ddos
7      normal             normal                   normal
8   injection          injection                injection
9    password           password                 password
10   scanning           scanning                 scanning
11   scanning           scanning                 scanning
12     normal             normal                   normal
13        dos                dos                      dos
14  injection          injection                injection
15     normal             normal                   normal
16        xss 

In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Encode object or category columns
for col in X.select_dtypes(include=['object', 'category']).columns:
    le_col = LabelEncoder()
    X[col] = le_col.fit_transform(X[col])

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scale the features (important for KNN)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train KNN model
knn_model = KNeighborsClassifier(n_neighbors=5, weights='uniform', metric='euclidean')
knn_model.fit(X_train_scaled, y_train)

# Evaluate
y_pred = knn_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=le.classes_)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

# Optional: Find optimal k value
def find_optimal_k(X_train, X_test, y_train, y_test, max_k=20):
    k_values = range(1, max_k + 1)
    accuracies = []
    
    for k in k_values:
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        accuracies.append(accuracy_score(y_test, y_pred))
    
    optimal_k = k_values[np.argmax(accuracies)]
    print(f"Optimal k value: {optimal_k}")
    print(f"Best accuracy: {max(accuracies):.4f}")
    
    return optimal_k, accuracies

# Find optimal k
optimal_k, accuracies = find_optimal_k(X_train_scaled, X_test_scaled, y_train, y_test)

# Train with optimal k
knn_optimal = KNeighborsClassifier(n_neighbors=optimal_k)
knn_optimal.fit(X_train_scaled, y_train)
y_pred_optimal = knn_optimal.predict(X_test_scaled)
accuracy_optimal = accuracy_score(y_test, y_pred_optimal)

print(f"\nOptimal KNN Results:")
print(f"Accuracy with k={optimal_k}: {accuracy_optimal}")
print("Classification Report:")
print(classification_report(y_test, y_pred_optimal, target_names=le.classes_))

NameError: name 'X' is not defined